# Coursera Spacecraft Dynamics and Control - Kinematics
## Module 3
## Concept Check 5 - Feedback Gain Selection

In [7]:
import numpy as np
from plotly import graph_objects as pgo
from plotly import express as px
import scipy
import pandas as pd
import sys

sys.path.append('../../')

import attitude_math as am

Consider the regulation problem where the unperturbed spacecraft is driven to the zero orientation.  The control law is: 
$u = -K\boldsymbol{\sigma}_{B/N} - [P]\boldsymbol{\omega}_{B/N} + [\boldsymbol{\omega}_{\tilde{B}/N}][I]\boldsymbol{\omega}_{B/N}$

The principal inertias are $I_1 = 100 \, \text{kg m}^2$, $I_2 = 75 \, \text{kg m}^2$, and $I_3 = 80 \, \text{kg m}^2$. Use the gains $K = 5 \, \text{Nm}$.

Choose a gain matrix such that $[P] = \text{diag}(P_1, P_2, P_3)$.

What gains $P_i$ will make the linearized closed-loop response critically damped with $\xi_i = 1$?


In [8]:
# adjust the return matrix values as needed
def result():
    K = 5
    I = [100, 75, 80]
    gainList = [(K*i)**0.5 for i in I] # P1, P2, P3
    return gainList
result()

[22.360679774997898, 19.364916731037084, 20.0]

### Question 2 setup
Generic Differential Kinematic function for question 2

In [9]:
def f(state, t, H_dot, I_inv):
    sigma = state[:3]
    omega = state[3:]

    sigma_dot = am.MRP_omega_DKE(sigma, omega)
    # Note in the below H_dot omega_R/N is zero and
    # the omega cross I*omega is cancelled
    omega_dot = I_inv @ H_dot
    state_dot = np.concatenate([sigma_dot, omega_dot])
    return state_dot

def Full_State_MRP_short_integrator(pdot,p0,t0,tf,dt):
    """
    Integrate quaternion rates to get new attitude quaternion
    """
    length=int((tf-t0)/dt)+1
    p=np.zeros([length,6])
    p[0,:]=np.array([p0])
    for i,t in enumerate(np.arange(t0, tf, dt)):
        p_new = p[i] + pdot(p[i],t).transpose()*dt
        p[i+1,:] = p_new
        p[i+1,:3] = am.MRP_short(p_new[:3])
    return p

### Question 2
Simulate the closed loop response with the initial states $\boldsymbol{\sigma}_{B/N}(t_0) = (0.1, 0.2, -0.1)$ and $\boldsymbol{\omega}_{B/N}(t_0) = (30, 10, -20)$ deg/sec. What is the tracking error MRP norm $|\boldsymbol{\sigma}_{B/R}|$ at 30 seconds into the simulation?

In [11]:
## Model description Properties
I = np.diag([100,75,80])
I_inv = np.linalg.inv(I)
K = 5
P = np.diag(np.array(result()))
sigma_0 = np.array([0.1, 0.2, -0.1])
omega_0 = np.deg2rad(np.array([30,10,-20]))
state_0 = np.concatenate([sigma_0,omega_0])
freq = 0.05

In [13]:
# Reference Tracking State Functions
sigma_r = lambda t, f :  np.zeros(3)
sigma_dot_r = lambda t, f : np.zeros(3)
def state_r(t, dt, f) : 
    sigma = sigma_r(t, f)
    sigma_dot = sigma_dot_r(t, f)
    sigma_old = sigma_r(t-dt, f)
    sigma_dot_old = sigma_dot_r(t-dt, f)

    omega_r = am.MRPdot_2_omega(sigma, sigma_dot)
    omega_r_old = am.MRPdot_2_omega(sigma_old, sigma_dot_old)

    omega_r_dot = (omega_r-omega_r_old)/dt
    return np.concatenate([sigma, omega_r, omega_r_dot])

# H_dot with regulator cancellations for speed up
def H_dot(state, state_r, I, K, P):
    sigma = state[:3]
    omega = state[3:]
    #att_diff = am.MRP_diff(state_r[:3],sigma) 
    att_diff = am.DCM_2_MRP(am.MRP_2_DCM(sigma)@am.MRP_2_DCM(state_r[:3]).transpose())
    omega_r_br = am.MRP_2_DCM(att_diff)@state_r[3:6]
    omega_rdot_br = am.MRP_2_DCM(att_diff)@state_r[6:]

    # Note the w X IW terms are cancelled and ignored below
    control = - K*att_diff - P@(omega-omega_r_br)
    return  control

# Integrate the MRP differential kinematic equations
t_step = .01
t_f = 60

dot = lambda state, t : f(state, t, H_dot(state, state_r(t, t_step, freq), I, K, P), I_inv)
#state_t = scipy.integrate.odeint(dot, state_0, np.arange(0., t_f, t_step))
state_t = Full_State_MRP_short_integrator(dot, state_0, 0., t_f, t_step)

# Calculate the MRP norm squared at 2 times
time_measure=30
answer_i  = int(time_measure/t_step)
del_sigma = am.DCM_2_MRP(am.MRP_2_DCM(state_t[answer_i,:3])@am.MRP_2_DCM(sigma_r(time_measure, freq)).transpose())
print(f"At Time: {time_measure}\tMRP B/R: {del_sigma}\tand its Norm: {np.linalg.norm(del_sigma)}")
time_measure=t_f
answer_i  = int(time_measure/t_step)
del_sigma = am.DCM_2_MRP(am.MRP_2_DCM(state_t[answer_i,:3])@am.MRP_2_DCM(sigma_r(time_measure, freq)).transpose())
print(f"At Time: {time_measure}\tMRP B/R: {del_sigma}\tand its Norm: {np.linalg.norm(del_sigma)}")

# Plots
fig = pgo.Figure()
sigma = np.array([sigma_r(t,freq) for t in np.arange(0., t_f, t_step)])
sigma_dot = np.array([sigma_dot_r(t,freq) for t in np.arange(0., t_f, t_step)])
fig.add_scatter3d(x=state_t[:,0], y=state_t[:,1], z=state_t[:,2],mode="lines")
fig.add_scatter3d(x=sigma[:,0],y=sigma[:,1],z=sigma[:,2],mode="lines")
fig.show()

state_data = np.concatenate([np.array(np.arange(0., t_f+t_step, t_step)).reshape(-1,1),state_t],axis=1)
state_data = pd.DataFrame(state_data, columns=["t","sigma1","sigma2","sigma3","omega1","omega2","omega3"])
px.line(state_data,x="t",y=state_data.columns[1:4]).show()

px.line(state_data,x="t",y=state_data.columns[4:]).show()

At Time: 30	MRP B/R: [ 0.08830766 -0.00036795 -0.09904829]	and its Norm: 0.13269869372127427
At Time: 60	MRP B/R: [ 0.00531442 -0.00050137 -0.00428328]	and its Norm: 0.006844040082605672


### Question 3
What are the corresponding decay times $T_i$ for each axis?


In [17]:
Is = I@np.ones(3)
[2*i/p for i,p in zip(Is,result())]

[8.94427190999916, 7.745966692414834, 8.0]